In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
import datetime
sns.set_theme(style='darkgrid')

In [ ]:
df = pd.read_csv("/content/drive/MyDrive/MASTER EN DATA SCIENCE/Capstone/data_dsmarket/GrupoRetail_1/Proceso Total/BBDD output/database_v5_floats.csv")

In [ ]:
df

In [ ]:
df.info()

In [ ]:
df.shape

In [ ]:
df

In [ ]:
df.isnull().sum()

In [ ]:
for i in ["date"]:
    df[i]=pd.to_datetime(df[i])

In [ ]:
df_sin_duplicados = df.copy()

In [ ]:
df_sin_duplicados=df.drop_duplicates().copy(deep=True)

In [ ]:
df_sin_duplicados.reset_index(drop=True, inplace=True)

In [ ]:
df_sin_duplicados.shape

In [ ]:
df_sin_duplicados.isnull().sum()

In [ ]:
# df_sin_duplicados['sales'].value_counts()

In [ ]:
df_sin_duplicados.info()

In [ ]:
df_sin_duplicados.isnull().sum()

In [ ]:
pd.to_numeric(df_sin_duplicados['sell_price'], errors='coerce')

In [ ]:
# len(df_sin_duplicados[df_sin_duplicados['sales'].isnull()])

In [ ]:
df_sinnulos = df_sin_duplicados.copy()

In [ ]:
df_sinnulos.dropna(inplace=True)

In [ ]:
df_sinnulos.isnull().sum()

In [ ]:
df_sinnulos.shape

In [ ]:
df_sinnulos.reset_index(drop=True, inplace=True)

In [ ]:
df_final = df_sinnulos.copy()

In [ ]:
df_final['sales'] = df_final['units'] * df_final['sell_price']

In [ ]:
df_final['year'] = df_final['year'].apply(int)

In [ ]:
df_final.head()

In [ ]:
del(df_final['Unnamed: 0'])

In [ ]:
df_final['week'] = df_final['week'].apply(int)

In [ ]:
df_final['month'] = pd.DatetimeIndex(df_final['date']).month

In [ ]:
df_final['day'] = pd.DatetimeIndex(df_final['date']).day

In [ ]:
df_final

In [ ]:
df_final.info()

In [ ]:
df_final.shape

In [ ]:
df_final['item'].unique()

In [ ]:
df_final.groupby(['item','store','year'])['sales'].sum()

In [ ]:
df_final

In [ ]:
weekend = pd.read_csv("/content/drive/MyDrive/MASTER EN DATA SCIENCE/Capstone/data_dsmarket/GrupoRetail_1/Proceso Total/BBDD output/database_weekends_events_4.csv")

In [ ]:
del(weekend["Unnamed: 0"])

In [ ]:
weekend

In [ ]:
df_final

In [ ]:
df_final = pd.merge(
    left = df_final,
    right = weekend,
    how = 'left',
    on = ["id", "year", "week"])

In [ ]:
df_final

In [ ]:
del(df_final["units_y"])

In [ ]:
df_final["units"] = df_final["units_x"]
del(df_final["units_x"])

In [ ]:
df_final = df_final.replace(np.nan, 0)

In [ ]:
df_final.isnull().sum()

In [ ]:
df_final

In [ ]:
df_final["summer_sales"] = np.where( (df_final.month == 6) |  (df_final.month == 7) |  (df_final.month == 8) , df_final.units, 0)

In [ ]:
df_final["christmas_sales"] = np.where( (df_final.month == 12) , df_final.units, 0)

In [ ]:
df_final_nan = df_final[ (df_final["units"]) == 0]
print(len(df_final))
print(len(df_final_nan))

In [ ]:
min_max_dates = df_final_nan.groupby(["item"]).agg({'date' : [np.min, np.max]}).reset_index()

In [ ]:
min_max_dates.columns = range(min_max_dates.shape[1])

In [ ]:
lista_1 = ["item", "min", "max"]
lista_dates = min_max_dates.columns.tolist()
dict_dates = dict(zip(lista_dates, lista_1))
min_max_dates.rename( columns =dict_dates, inplace = True)

In [ ]:
df_final.info()

In [ ]:
precio_inicio = df_final.groupby(["item", "date"]).max("sell_price").reset_index()

In [ ]:
precio_inicio["min"] = precio_inicio["date"]

In [ ]:
precio_inicio = precio_inicio[["item", "min", "sell_price"]]

In [ ]:
precios = pd.merge(
    left = min_max_dates,
    right = precio_inicio,
    how = 'left',
    on = ["item", "min"])

In [ ]:
precios.rename(columns = {"sell_price" : "precio_inicio" })

In [ ]:
precios['exhibition_time'] = (precios['max'] - precios['min']).dt.days

In [ ]:
items_sales = df_final.groupby("item").sum()

In [ ]:
items_sales

Discount

In [ ]:
from datetime import datetime, timedelta

In [ ]:
last_week = df_final.groupby(["date", "item"]).min()
last_week.reset_index(inplace = True)

In [ ]:
last_week["date_last_week"] = last_week["date"] - timedelta(days = 7 )

In [ ]:
lista = last_week.columns.tolist()
lista
lista.remove("date")
lista.remove("item")
lista.remove("date_last_week")
lista.remove("sell_price")
for i in lista:
    del(last_week[i])

In [ ]:
discount = df_final.groupby(["date", "item"]).min()
discount.reset_index (inplace = True)

In [ ]:
lista = discount.columns.tolist()
lista
lista.remove("date")
lista.remove("item")
lista.remove("sell_price")
for i in lista:
    del(discount[i])

In [ ]:
discount.rename(columns = {"date" : "date_last_week" }, inplace=True)

In [ ]:
first_discount = pd.merge(
    left = last_week,
    right = discount,
    how = 'left',
    on = ["date_last_week", "item"])

In [ ]:
first_discount.rename(columns = {"sell_price_y" : "precio_last_week" }, inplace=True)

In [ ]:
first_discount["cambio"] = first_discount["precio_last_week"]- first_discount["sell_price_x"]

In [ ]:
first_discount.groupby("cambio").first()
first_discount.reset_index(inplace = True)

In [ ]:
first_discount["discount"] = first_discount["precio_last_week"] / first_discount["sell_price_x"]-1

In [ ]:
first_discount["discount"] = np.where( (first_discount.discount < -0.05) , first_discount.discount, 0)

In [ ]:
first_discount = first_discount[ (first_discount["discount"]) != 0]

In [ ]:
first_discount = first_discount.sort_values(by="date").groupby("item").first()

In [ ]:
first_discount.reset_index (inplace = True)
lista = ["date", "sell_price_x", "cambio", "index"]
for i in lista:
    del(first_discount[i])

In [ ]:
precios_discounts = pd.merge(
    left = precios,
    right = first_discount,
    how = 'left',
    on = ["item"])

In [ ]:
precios_discounts['days_until_first_discount'] = (precios_discounts['date_last_week'] - precios_discounts['min']).dt.days

In [ ]:
precios_discounts

In [ ]:
precio_first = precios_discounts.groupby("item").first("sell.price")

In [ ]:
del(precio_first["exhibition_time"])
del(precio_first["precio_last_week"])
del(precio_first["discount"])
del(precio_first["days_until_first_discount"])
precio_first.rename(columns = {"sell_price" : "precio_inicio" }, inplace=True)

In [ ]:
dias_fp = pd.merge(
    left = precio_inicio,
    right = precio_first,
    how = 'left',
    on = ["item"])

In [ ]:
dias_fp["discount_on_date"] = dias_fp["sell_price"]- dias_fp["precio_inicio"]

In [ ]:
dias_fp["full_price_dummy"] = np.where( (dias_fp.discount_on_date == 0) , 1, 0)

In [ ]:
fp_days = dias_fp.groupby("item").sum()
fp_days.reset_index(inplace =True)

In [ ]:
fp_days

In [ ]:
del(fp_days["sell_price"])
del(fp_days["precio_inicio"])
del(fp_days["discount_on_date"])

In [ ]:
precios_discount_fp = pd.merge(
    left = precios_discounts,
    right = fp_days,
    how = 'left',
    on = ["item"])

In [ ]:
del(precios_discount_fp["date_last_week"])
del(precios_discount_fp["precio_last_week"])
precios_discount_fp.rename(columns = {"sell_price" : "precio_inicio", "min" : "fecha_inicio_venta", "max": "fecha_fin_venta", "discount" : "first_discount"  }, inplace=True)

In [ ]:
precios_discount_fp

In [ ]:
df_final["sales_2011"] = np.where( (df_final.year == 2011) , df_final.units, 0)
df_final["sales_2012"] = np.where( (df_final.year == 2012) , df_final.units, 0)
df_final["sales_2013"] = np.where( (df_final.year == 2013) , df_final.units, 0)
df_final["sales_2014"] = np.where( (df_final.year == 2014) , df_final.units, 0)
df_final["sales_2015"] = np.where( (df_final.year == 2015) , df_final.units, 0)
df_final["sales_2016"] = np.where( (df_final.year == 2016) , df_final.units, 0)

In [ ]:
df_final.info()

In [ ]:
df_final.to_csv("/content/drive/MyDrive/MASTER EN DATA SCIENCE/Capstone/data_dsmarket/GrupoRetail_1/Proceso Total/BBDD output/DF_FINAL.csv")  # este archivo va al sales forecast

In [ ]:
items_sales = df_final.groupby(["item","category"]).sum()
items_sales.reset_index(inplace = True)
list = ["year", "week", "month", "day", "weekday_int"]
for i in list:
    del(items_sales[i])

In [ ]:
items_sales["mean_pvp"] = items_sales["sales"] / items_sales ["units"]

In [ ]:
items_sales["weekend_sale_total"] = items_sales["weekend_sales"] / items_sales ["sales"]

In [ ]:
items_sales["event_sales_total"] = items_sales["event_sales"] / items_sales ["sales"]

In [ ]:
items_sales["summer_sales_total"] = items_sales["summer_sales"] / items_sales ["sales"]

In [ ]:
items_sales["christmas_sales_total"] = items_sales["christmas_sales"] / items_sales ["sales"]

In [ ]:
items_sales["2015_vs_2014"] = items_sales["sales_2015"] / items_sales ["sales_2014"]-1

In [ ]:
items_sales["2015_vs_2013"] = items_sales["sales_2015"] / items_sales ["sales_2014"]-1

In [ ]:
items_sales

In [ ]:
df_cluster = pd.merge(
    left = items_sales,
    right = precios_discount_fp,
    how = 'left',
    on = ["item"])

In [ ]:
df_cluster["full_price_sales"] = df_cluster["precio_inicio"] * df_cluster["units"]
df_cluster["mean_discount"] = df_cluster["sales"] / df_cluster["full_price_sales"]-1

In [ ]:
df_cluster

In [ ]:
del(df_cluster["sell_price"])
del(df_cluster["weekend_sales"])
del(df_cluster["event_sales"])
del(df_cluster["summer_sales"])
del(df_cluster["sales_2015"])
del(df_cluster["sales_2011"])
del(df_cluster["sales_2014"])
del(df_cluster["sales_2013"])
del(df_cluster["sales_2012"])
del(df_cluster["sales_2016"])
del(df_cluster["fecha_inicio_venta"])
del(df_cluster["fecha_fin_venta"])
del(df_cluster["christmas_sales"])
del(df_cluster["event_dummy"])

In [ ]:
df_cluster

In [ ]:
df_cluster.to_excel("/content/drive/MyDrive/MASTER EN DATA SCIENCE/Capstone/data_dsmarket/GrupoRetail_1/Proceso Total/BBDD output/bd_clustering.xlsx")

In [ ]:
# EL FICHERO bd_clustering SE MODIFICARÁ AÑADIENDO MÁS DATOS PASANDO A SER EL FICHERO bbdd_clustering_v1 QUE SE ENCUENTRA EN EL DIRECTORIO OUTPUT